In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score  #交叉验证评分
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error #均方根误差
from sklearn.metrics import roc_curve, auc, recall_score, precision_score
# 导入数据，路径中要么用\\或/或者在路径前加r
dataset = pd.read_csv('../data/train_chuli.csv')

In [32]:
# 输出数据预览
print(dataset.head())

   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare  \
0            1         0       3    1  22.0      1      0   7.2500   
1            2         1       1    0  38.0      1      0  71.2833   
2            3         1       3    0  26.0      0      0   7.9250   
3            4         1       1    0  35.0      1      0  53.1000   
4            5         0       3    1  35.0      0      0   8.0500   

   family_size  Embarked_C  Embarked_Q  Embarked_S  
0            2           0           0           1  
1            2           1           0           0  
2            1           0           0           1  
3            2           0           0           1  
4            1           0           0           1  


In [33]:
from sklearn.model_selection import train_test_split #划分数据集
# 准备训练数据
x = dataset.drop(['Survived'], axis = 1)
y = dataset['Survived']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0) 
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(623, 11) (268, 11) (623,) (268,)


In [34]:
from matplotlib.cm import rainbow 
from sklearn.naive_bayes import GaussianNB 
#
rf_scores = []					# 如果bootstrap为True，则从X抽取以训练每个决策树。
rf_classifier =  GaussianNB (var_smoothing=1e-10)
rf_classifier.fit(x_train, y_train)

GaussianNB(var_smoothing=1e-10)

GaussianNB
- 先验概率(prior)：指类别的先验概率分布，即不考虑任何数据的情况下，每个类别出现的概率。在GaussianNB中，默认情况下，每个类别的先验概率都是相等的，可以通过设置prior参数来改变先验概率。如果有领域知识可以确定类别的先验概率，可以通过调整prior参数来提高模型的准确性。
- 方差(var_smoothing)：在计算概率密度时，为了避免出现0的情况，会将所有特征的方差加上一个小的数值(var_smoothing)。默认情况下，var_smoothing=1e-9，可以通过调整var_smoothing参数来改变方差的大小。

In [35]:
#评价指标
import numpy as np

# 计算混淆矩阵
def compute_confusion_matrix(precited,expected):
    part = precited ^ expected             # 对结果进行分类，亦或使得判断正确的为0,判断错误的为1
    pcount = np.bincount(part)             # 分类结果统计，pcount[0]为0的个数，pcount[1]为1的个数
    tp_list = list(precited & expected)    # 将TP的计算结果转换为list
    fp_list = list(precited & ~expected)   # 将FP的计算结果转换为list
    tp = tp_list.count(1)                  # 统计TP的个数
    fp = fp_list.count(1)                  # 统计FP的个数
    tn = pcount[0] - tp                    # 统计TN的个数
    fn = pcount[1] - fp                    # 统计FN的个数
    return tp, fp, tn, fn

# 计算常用指标
def compute_indexes(tp, fp, tn, fn):
    accuracy = (tp+tn) / (tp+tn+fp+fn)     # 准确率
    precision = tp / (tp+fp)               # 精确率
    recall = tp / (tp+fn)                  # 召回率
    F1 = (2*precision*recall) / (precision+recall)    # F1
    return accuracy, precision, recall, F1


In [36]:
rfc_score = rf_classifier.score(x_test, y_test)#测试集准确率,j即accuracy
expected = y_test.astype('int32')#预期值
# print(type(expected))
# print(expected)
rfc_pre=rf_classifier.predict(x_test)
precited = rfc_pre.astype('int32')#模型预测值
# print(type(precited))
tp, fp, tn, fn = compute_confusion_matrix(precited, expected)
accuracy, precision, recall, F1 = compute_indexes(tp, fp, tn, fn)
rfc_pre = rf_classifier.predict(x_test)
rfc_auc = metrics.roc_auc_score(y_test, rfc_pre)
rfc_rmse = np.sqrt(mean_squared_error(y_test, rfc_pre))
print('TP FP TN FN')
print('%d %d %d %d'%(tp,fp,tn,fn))
print('accuracy precision recall F1')
print('%.4lf %.4lf %.4lf %.4lf'%(accuracy, precision, recall, F1))
print('auc rmse')
print('%.4lf %.4lf'%(rfc_auc, rfc_rmse))

TP FP TN FN
79 47 121 21
accuracy precision recall F1
0.7463 0.6270 0.7900 0.6991
auc rmse
0.7551 0.5037
